In [ ]:
import pandas as pd
import math
from openpyxl import load_workbook
import pyodbc
from datetime import datetime, timedelta
import numpy

FILENAME = 'MDC Q1 Playbook Lam WEEKLY UPDATES.xlsx'
SHEET_NAME = 'MDC Supply Commits'
UPDATED_SHEET_NAME = SHEET_NAME + ' - updated'
FAIR_SHEET_NAME = 'MDC Q1 Playbook'

CONNECTION_STRING = f'\
            DRIVER={{ODBC Driver 17 for SQL Server}};\
            ENCRYPT=no;\
            SERVER=172.20.103.105;\
            DATABASE=MDC_AX2009SP1_PROD;\
            UID=USSQLUser;\
            PWD=Insp1r0n2022'

In [ ]:
commits_orig = pd.read_excel(FILENAME, sheet_name=SHEET_NAME)
commits_orig.insert(1, 'Type', '')

# clear out existing values
commits_orig.iloc[:, 5:-1] = numpy.nan

fairs = pd.read_excel(FILENAME, FAIR_SHEET_NAME)


commits_orig[commits_orig['PartNumber'] == '10-143502-00']

In [ ]:
def isNaN(value):
    return isinstance(value, float) and math.isnan(value)

def exec_query(query):
    cursor = pyodbc.connect(CONNECTION_STRING).cursor()
    cursor.execute(query)
    rows = [tuple(r) for r in cursor.fetchall()]
    headers = [column[0] for column in cursor.description]
    data = [dict(zip(headers, row)) for row in rows]
    return data

In [ ]:
SO_LINES_QUERY = '''
        SELECT ITEMID, CAST(SALESQTY AS INT) AS SALESQTY, SHIPPINGDATECONFIRMED
        FROM [MDC_AX2009SP1_PROD].[dbo].[SALESLINE]
        WHERE DATAAREAID='mdc'
        '''

so_lines = pd.DataFrame(exec_query(SO_LINES_QUERY))
so_lines

In [ ]:
sums = so_lines.loc[so_lines['SHIPPINGDATECONFIRMED'] >= '2023-06-01'].groupby('ITEMID')['SALESQTY'].sum()
sums

In [ ]:
commits = commits_orig.copy()
commits = commits[commits['PartNumber'].notnull()] # filter out rows with empty part numbers
commits['Type'] = 'Supply Commits'
commits

In [ ]:
date_cols = [col for col in commits.columns if isinstance(col, datetime)]
date_cols

In [ ]:
for date_col in date_cols:
    commits[date_col] = commits['PartNumber'].map(
        so_lines.loc[
            (so_lines['SHIPPINGDATECONFIRMED'] >= date_col)
            & (so_lines['SHIPPINGDATECONFIRMED'] < date_col + timedelta(days=7))
        ].groupby('ITEMID')['SALESQTY'].sum()
    )

commits


In [ ]:
commits.iloc[:, 5] = commits['PartNumber'].map(
    so_lines.loc[
        (so_lines['SHIPPINGDATECONFIRMED'] < date_cols[0])
    ].groupby('ITEMID')['SALESQTY'].sum()
)

commits[commits.iloc[:, 5].notnull()]

In [ ]:
shipped = commits_orig.copy()
shipped = shipped[shipped['PartNumber'].notnull()] # filter out rows with empty part numbers
shipped['Type'] = 'Completed Shipments'
shipped['Issues/Notes'] = ''
shipped

In [ ]:
SPS_LINES_QUERY = '''
        SELECT ITEMID, CAST(QTY AS INT) AS QTY, DELIVERYDATE
        FROM [MDC_AX2009SP1_PROD].[dbo].[CUSTPACKINGSLIPTRANS]
        WHERE DATAAREAID='mdc'
        '''

sps_lines = pd.DataFrame(exec_query(SPS_LINES_QUERY))
sps_lines

In [ ]:
sps_lines[sps_lines['ITEMID'] == '839-298117-001']

In [ ]:
for date_col in date_cols:
    shipped[date_col] = shipped['PartNumber'].map(
        sps_lines.loc[
            (sps_lines['DELIVERYDATE'] >= date_col)
            & (sps_lines['DELIVERYDATE'] < date_col + timedelta(days=7))
        ].groupby('ITEMID')['QTY'].sum()
    )


In [ ]:
shipped.iloc[:, 5] = shipped['PartNumber'].map(
    sps_lines.loc[
        (sps_lines['DELIVERYDATE'] < date_cols[0])
    ].groupby('ITEMID')['QTY'].sum()
)

shipped[shipped['PartNumber'] == '839-298117-001']

In [ ]:
combined = pd.concat([commits, shipped]).sort_values(['PartNumber', 'Type'], ascending=False)
combined

In [ ]:
past_thru_col = combined.columns[5]
totals_col = combined.columns[-2]

combined[totals_col] = combined[[past_thru_col] + date_cols].sum(axis=1)

combined[past_thru_col] = combined[past_thru_col].replace(0, numpy.nan)
combined

In [ ]:

fairs_filtered = fairs[fairs['Group#'] == 'Group 2'][['Lam Part No', 'FAIR Date']]

combined_merged = pd.merge(combined, fairs_filtered, left_on='PartNumber', right_on='Lam Part No', how='left')

combined_merged['FAIR Completed Date'] = combined_merged['FAIR Date']
combined_merged = combined_merged.drop(columns=['Lam Part No', 'FAIR Date'])

In [ ]:
shipped[shipped['PartNumber'] == '10-143502-00']

In [ ]:
book = load_workbook(FILENAME)

writer = pd.ExcelWriter(FILENAME, engine='openpyxl')
writer.book = book

combined_merged.to_excel(writer, sheet_name=UPDATED_SHEET_NAME, index=False)
writer.save()
writer.close()
